In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer

# Load the data
X_train = pd.read_csv('data/X_train.csv')
Y_train = pd.read_csv('data/Y_train.csv')
X_test = pd.read_csv('data/X_test.csv')

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_8060\3181913993.py:14: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  X_train = pd.read_csv('data/X_train.csv')


In [31]:
# x_train.isnull().sum()
# x_train.info()



In [32]:
# for row in range(x_train.shape[1]):
#     if(x_train.iloc[:,row].dtype=="O"):
#         print("{}: {}\n".format(x_train.columns[row],x_train.iloc[:,row].unique()))

In [34]:
X_train.drop(['ID'],axis=1,inplace=True)
# Xử lý dữ liệu thiếu cho các cột số
numeric_columns = X_train.select_dtypes(include=['int64', 'float64']).columns
imputer = SimpleImputer(strategy='mean')
X_train[numeric_columns] = imputer.fit_transform(X_train[numeric_columns])
X_test[numeric_columns] = imputer.transform(X_test[numeric_columns])

# Xử lý dữ liệu thiếu cho các cột categorical
categorical_columns = X_train.select_dtypes(include=['object']).columns
X_train[categorical_columns] = X_train[categorical_columns].fillna('Unknown')
X_test[categorical_columns] = X_test[categorical_columns].fillna('Unknown')

In [35]:
# Chuyển đổi cột tradeTime thành datetime
X_train['tradeTime'] = pd.to_datetime(X_train['tradeTime'])
X_test['tradeTime'] = pd.to_datetime(X_test['tradeTime'])

# Tạo các tính năng mới từ tradeTime
X_train['trade_year'] = X_train['tradeTime'].dt.year
X_train['trade_month'] = X_train['tradeTime'].dt.month
X_train['trade_day'] = X_train['tradeTime'].dt.day

X_test['trade_year'] = X_test['tradeTime'].dt.year
X_test['trade_month'] = X_test['tradeTime'].dt.month
X_test['trade_day'] = X_test['tradeTime'].dt.day

# Xử lý các cột categorical khác
for col in ['livingRoom', 'drawingRoom', 'bathRoom', 'floor', 'constructionTime']:
    X_train[col] = X_train[col].astype('category')
    X_test[col] = X_test[col].astype('category')
    
    # Sử dụng one-hot encoding
    X_train = pd.get_dummies(X_train, columns=[col], prefix=col)
    X_test = pd.get_dummies(X_test, columns=[col], prefix=col)

# Đảm bảo X_test có cùng các cột như X_train
missing_cols = set(X_train.columns) - set(X_test.columns)
for col in missing_cols:
    X_test[col] = 0
X_test = X_test[X_train.columns]

In [ ]:
# def remove_outliers(df, columns, z_score_threshold=3):
#     for col in columns:
#         z_scores = np.abs((df[col] - df[col].mean()) / df[col].std())
#         df = df[z_scores < z_score_threshold]
#     return df

# numeric_columns = X_train.select_dtypes(include=['int64', 'float64']).columns
# X_train = remove_outliers(X_train, numeric_columns)
# Y_train = Y_train.loc[X_train.index]
# from sklearn.feature_selection import VarianceThreshold

# # Loại bỏ các tính năng có phương sai thấp
# selector = VarianceThreshold(threshold=0.01)
# X_train_selected = selector.fit_transform(X_train)
# X_test_selected = selector.transform(X_test)

# # Cập nhật DataFrame
# X_train = pd.DataFrame(X_train_selected, columns=X_train.columns[selector.get_support()])
# X_test = pd.DataFrame(X_test_selected, columns=X_test.columns[selector.get_support()])

In [36]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numeric_columns = X_train.select_dtypes(include=['int64', 'float64']).columns
X_train[numeric_columns] = scaler.fit_transform(X_train[numeric_columns])
X_test[numeric_columns] = scaler.transform(X_test[numeric_columns])

In [37]:
X_train.to_csv('data/X_train_processed.csv', index=False)
X_test.to_csv('data/X_test_processed.csv', index=False)
Y_train.to_csv('data/Y_train_processed.csv', index=False)

In [42]:
# Đọc dữ liệu
X_train = pd.read_csv('data/X_train_processed.csv')
X_test = pd.read_csv('data/X_test_processed.csv')
Y_train = pd.read_csv('data/Y_train_processed.csv')

# Chuyển đổi cột 'tradeTime' thành datetime
X_train['tradeTime'] = pd.to_datetime(X_train['tradeTime'])
X_test['tradeTime'] = pd.to_datetime(X_test['tradeTime'])

# Tạo các tính năng mới từ 'tradeTime'
X_train['trade_year'] = X_train['tradeTime'].dt.year
X_train['trade_month'] = X_train['tradeTime'].dt.month
X_train['trade_day'] = X_train['tradeTime'].dt.day

X_test['trade_year'] = X_test['tradeTime'].dt.year
X_test['trade_month'] = X_test['tradeTime'].dt.month
X_test['trade_day'] = X_test['tradeTime'].dt.day

# Xóa cột 'tradeTime' gốc
X_train = X_train.drop('tradeTime', axis=1)
X_test = X_test.drop('tradeTime', axis=1)

# Chia dữ liệu thành tập train và validation
X_train, X_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)


In [43]:

# Huấn luyện mô hình RandomForest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Dự đoán bằng RandomForest
rf_train_pred = rf_model.predict(X_train)
rf_val_pred = rf_model.predict(X_val)

# Xây dựng mô hình MLP
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1)
])

d:\App\anacoda\envs\ai_class\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [44]:
# Compile mô hình
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Callback để giảm learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# Huấn luyện mô hình MLP với dự đoán của RandomForest làm target
history = model.fit(
    X_train, rf_train_pred,
    validation_data=(X_val, rf_val_pred),
    epochs=100,
    batch_size=32,
    callbacks=[reduce_lr],
    verbose=1
)


Epoch 1/100
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 47s 7ms/step - loss: 10662230016.0000 - val_loss: 9921668096.0000 - learning_rate: 0.0010
Epoch 2/100
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 45s 7ms/step - loss: 10007766016.0000 - val_loss: 9833856000.0000 - learning_rate: 0.0010
Epoch 3/100
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 46s 7ms/step - loss: 9970583552.0000 - val_loss: 9867708416.0000 - learning_rate: 0.0010
Epoch 4/100
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 47s 7ms/step - loss: 9935080448.0000 - val_loss: 9732967424.0000 - learning_rate: 0.0010
Epoch 5/100
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 41s 6ms/step - loss: 9872155648.0000 - val_loss: 9685792768.0000 - learning_rate: 0.0010
Epoch 6/100
3423/6377 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - loss: 9827852288.0000

KeyboardInterrupt: 

In [ ]:
# Đánh giá mô hình MLP
mlp_train_pred = model.predict(X_train)
mlp_val_pred = model.predict(X_val)

# Tính MSE và R2 cho RandomForest
rf_train_mse = mean_squared_error(y_train, rf_train_pred)
rf_val_mse = mean_squared_error(y_val, rf_val_pred)
rf_train_r2 = r2_score(y_train, rf_train_pred)
rf_val_r2 = r2_score(y_val, rf_val_pred)

# Tính MSE và R2 cho MLP
mlp_train_mse = mean_squared_error(y_train, mlp_train_pred)
mlp_val_mse = mean_squared_error(y_val, mlp_val_pred)
mlp_train_r2 = r2_score(y_train, mlp_train_pred)
mlp_val_r2 = r2_score(y_val, mlp_val_pred)

print("RandomForest - Train MSE:", rf_train_mse)
print("RandomForest - Validation MSE:", rf_val_mse)
print("RandomForest - Train R2:", rf_train_r2)
print("RandomForest - Validation R2:", rf_val_r2)

print("MLP - Train MSE:", mlp_train_mse)
print("MLP - Validation MSE:", mlp_val_mse)
print("MLP - Train R2:", mlp_train_r2)
print("MLP - Validation R2:", mlp_val_r2)

In [ ]:
# Dự đoán trên tập test
test_pred = model.predict(X_test)

# Lưu kết quả dự đoán
results = pd.DataFrame({'ID': range(len(test_pred)), 'TARGET': test_pred.flatten()})
results.to_csv('MLP_predictions.csv', index=False)

print("Kết quả đã được lưu vào file 'MLP_predictions.csv'")